# Impact of KCs children on the KC mastery  
In this notebook we will study a toy example of a 3 KCs domain knowledge model. The purpose of this example is to understand the impact children have on their parents.

## Domain knowledge model
We denote A, B and C the three studied KCs and we will suppose they are related with two prerequisite links A -> B and A -> C.

In [1]:
import sys
sys.path.append("/Users/olivier/PycharmProjects/bayesian-kst/")  # for mac
sys.path.append("/home/olivier/PycharmProjects/bayesian-kst/")  # for ubuntu

from kgraph.expert_layer.domain import Domain
from kgraph.expert_layer.knowledge_components import KnowledgeComponent
from kgraph.expert_layer.link import Link
from kgraph.resources_layer.exercise import Exercise
from kgraph.learner_layer.answer import LearnerAnswer
from kgraph.learner_layer.learner import Learner
from kgraph.learner_layer.learner_pool import LearnerPool
from kgraph.helpers.truthtable import truthtable
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.dynamicBN as gdyn
import random
import itertools
import numpy as np
from sklearn.metrics import roc_auc_score

KC_A = KnowledgeComponent(1, "A")
KC_B = KnowledgeComponent(2, "B")
KC_C = KnowledgeComponent(3, "C")

A_2_B = Link(source=KC_A, target=KC_B)
A_2_C = Link(source=KC_A, target=KC_C)

domain = Domain([KC_A, KC_B, KC_C], [A_2_B, A_2_C])

params = {"slip": .01, "guess":.01}

# there are 5 exercises corresponding to KC A
ex_1 = Exercise(1, KC_A, "qcm", ex_content="", params=params)
ex_2 = Exercise(2, KC_B, "qcm", ex_content="", params=params)
ex_3 = Exercise(3, KC_C, "qcm", ex_content="", params=params)

priors = {'A': 0.1, 'B': 0.1, 'C': 0.1}


## Learner knowledge model
### No prerequisite links
In a first place, in order to differenciate the effect of evaluations and the impact of the prerequisite links, we study how the model behave to learner traces when we don't declare any prerequisite links.

In [2]:
prereqs = {KC_A: {KC_B: 'not existing', KC_C:'not existing'}, 
           KC_B: {KC_A: 'not existing'}, 
           KC_C: {KC_A: 'not existing'}}

learner_pool = LearnerPool(domain, prereqs)

for kc in learner_pool.get_knowledge_components():
    learner_pool.set_learn(kc, .1)
    learner_pool.set_prior(kc, .1)
    learner_pool.set_slip(kc, .1)
    learner_pool.set_guess(kc, .25)
    learner_pool.set_forget(kc, 0)

In [3]:
learner = Learner(1, learner_pool)
answer = [KC_A, True]

print("Results from a succeeded exercise done on KC A:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

Results from a succeeded exercise done on KC A:

The knowledge state before doing the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.1}
The knowledge state after the exercise is:  {'A': 0.41304347826086957, 'B': 0.1, 'C': 0.1, 'eval(A)': 0.5184782608695652, 'eval(B)': 0.315, 'eval(C)': 0.315}


In [4]:
learner = Learner(1, learner_pool)
answer = [KC_A, False]

print("Results from an unsuccueeded exercise done on KC A:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

Results from an unsuccueeded exercise done on KC A:

The knowledge state before doing the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.1}
The knowledge state after the exercise is:  {'A': 0.02540106951871658, 'B': 0.1, 'C': 0.1, 'eval(A)': 0.26651069518716575, 'eval(B)': 0.315, 'eval(C)': 0.315}


In [5]:
learner = Learner(1, learner_pool)
answer = [KC_B, True]

print("Results from a succeeded exercise done on KC B:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

Results from a succeeded exercise done on KC B:

The knowledge state before doing the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.1}
The knowledge state after the exercise is:  {'A': 0.1, 'B': 0.41304347826086957, 'C': 0.1, 'eval(A)': 0.315, 'eval(B)': 0.5184782608695652, 'eval(C)': 0.315}


In [6]:
learner = Learner(1, learner_pool)
answer = [KC_B, False]

print("Results from an unsuccueeded exercise done on KC B:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

Results from an unsuccueeded exercise done on KC B:

The knowledge state before doing the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.1}
The knowledge state after the exercise is:  {'A': 0.1, 'B': 0.02540106951871658, 'C': 0.1, 'eval(A)': 0.315, 'eval(B)': 0.26651069518716575, 'eval(C)': 0.315}


In [7]:
learner = Learner(1, learner_pool)
answer = [KC_C, True]

print("Results from a succeeded exercise done on KC C:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

Results from a succeeded exercise done on KC C:

The knowledge state before doing the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.1}
The knowledge state after the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.41304347826086957, 'eval(A)': 0.315, 'eval(B)': 0.315, 'eval(C)': 0.5184782608695652}


In [8]:
learner = Learner(1, learner_pool)
answer = [KC_C, False]

print("Results from an unsuccueeded exercise done on KC C:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

Results from an unsuccueeded exercise done on KC C:

The knowledge state before doing the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.1}
The knowledge state after the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.02540106951871658, 'eval(A)': 0.315, 'eval(B)': 0.315, 'eval(C)': 0.26651069518716575}


### Strong prerequisite links
In a first place, in order to differenciate the effect of evaluations and the impact of the prerequisite links, we study how the model behave to learner traces when we don't declare any prerequisite links.

In [9]:
prereqs = {KC_A: {KC_B: 'strong', KC_C:'weak'}, 
           KC_B: {KC_A: 'strong'}, 
           KC_C: {KC_A: 'weak'}}
learner_pool = LearnerPool(domain, prereqs)

for kc in learner_pool.get_knowledge_components():
    learner_pool.set_learn(kc, .1)
    learner_pool.set_prior(kc, .1)
    learner_pool.set_slip(kc, .1)
    learner_pool.set_guess(kc, .25)
    learner_pool.set_forget(kc, 0)

In [10]:
learner = Learner(1, learner_pool)
priors = learner.get_priors()
answer = [KC_A, True]

print("Results from a succeeded exercise done on KC A:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

Results from a succeeded exercise done on KC A:

The knowledge state before doing the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.1}
The knowledge state after the exercise is:  {'A': 0.5747594374537379, 'B': 0.1739452257586973, 'C': 0.15396002960769795, 'eval(A)': 0.6235936343449296, 'eval(B)': 0.3630643967431532, 'eval(C)': 0.3500740192450037}


In [11]:
learner = Learner(1, learner_pool)
priors = learner.get_priors()
answer = [KC_A, False]

print("Results from an unsuccueeded exercise done on KC A:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

Results from an unsuccueeded exercise done on KC A:

The knowledge state before doing the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.1}
The knowledge state after the exercise is:  {'A': 0.04767313359528488, 'B': 0.026399803536345773, 'C': 0.046291748526522604, 'eval(A)': 0.28098753683693517, 'eval(B)': 0.26715987229862476, 'eval(C)': 0.2800896365422397}


In [12]:
learner = Learner(1, learner_pool)
priors = learner.get_priors()
answer = [KC_B, True]

print("Results from a succeeded exercise done on KC B:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

Results from a succeeded exercise done on KC B:

The knowledge state before doing the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.1}
The knowledge state after the exercise is:  {'A': 0.3706637297376235, 'B': 0.28304045173538434, 'C': 0.1, 'eval(A)': 0.4909314243294553, 'eval(B)': 0.43397629362799983, 'eval(C)': 0.31499999999999995}


In [13]:
learner = Learner(1, learner_pool)
priors = learner.get_priors()
answer = [KC_B, False]

print("Results from an unsuccueeded exercise done on KC B:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

Results from an unsuccueeded exercise done on KC B:

The knowledge state before doing the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.1}
The knowledge state after the exercise is:  {'A': 0.16621084107424336, 'B': 0.014410732519752951, 'C': 0.1, 'eval(A)': 0.3580370466982582, 'eval(B)': 0.2593669761378395, 'eval(C)': 0.315}


In [14]:
learner = Learner(1, learner_pool)
priors = learner.get_priors()
answer = [KC_C, True]

print("Results from a succeeded exercise done on KC C:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

Results from a succeeded exercise done on KC C:

The knowledge state before doing the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.1}
The knowledge state after the exercise is:  {'A': 0.34424288310115075, 'B': 0.09999999999999998, 'C': 0.2886129618413083, 'eval(A)': 0.47375787401574804, 'eval(B)': 0.31499999999999995, 'eval(C)': 0.4375984251968504}


In [15]:
learner = Learner(1, learner_pool)
priors = learner.get_priors()
answer = [KC_C, False]

print("Results from an unsuccueeded exercise done on KC C:\n")
print("The knowledge state before doing the exercise is: ",priors)
knowledge_state = learner.predict_next_step(priors, evaluation=answer, pred_mode='one_kc')
#knowledge_state = learner.predict_next_step(knowledge_state, evaluation=None, pred_mode='all')
print("The knowledge state after the exercise is: ",knowledge_state)

Results from an unsuccueeded exercise done on KC C:

The knowledge state before doing the exercise is:  {'A': 0.1, 'B': 0.1, 'C': 0.1}
The knowledge state after the exercise is:  {'A': 0.18242683608798316, 'B': 0.09999999999999999, 'C': 0.01480365353547906, 'eval(A)': 0.368577443457189, 'eval(B)': 0.315, 'eval(C)': 0.2596223747980614}
